In [23]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import timm
from tqdm import tqdm

# --- Config ---
IMG_SIZE = 224
BATCH_SIZE = 128
EPOCHS = 25
LEARNING_RATE = 3e-4
NUM_CLASSES = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAVE_PATH = "best_convnext_model.pth"
PATIENCE = 7

# Label mapping
label2idx = {'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'Red soil': 3}

# --- Model ---
model = timm.create_model('convnext_tiny', pretrained=True, num_classes=NUM_CLASSES, drop_rate=0.4)
model = model.to(DEVICE)

# --- Class Weights ---
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df.soil_type), y=df.soil_type)
weights = torch.tensor([class_weights[label2idx[k]] for k in label2idx], dtype=torch.float32).to(DEVICE)

# --- Loss, Optimizer, Scheduler ---
criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
scaler = GradScaler()

# --- Optional: MixUp Function ---
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(DEVICE)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# --- Training Loop ---
best_val_acc = 0
early_stop_counter = 0

for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss, train_correct = 0, 0
    train_preds, train_labels = [], []

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        # Apply MixUp
        images, targets_a, targets_b, lam = mixup_data(images, labels, alpha=0.4)

        with autocast():
            outputs = model(images)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        preds = outputs.argmax(1)
        # For accuracy, use targets_a
        train_loss += loss.item() * images.size(0)
        train_correct += (preds == targets_a).sum().item()
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(targets_a.cpu().numpy())

    train_acc = train_correct / len(train_loader.dataset)
    train_f1 = f1_score(train_labels, train_preds, average='weighted')

    # --- Validation ---
    model.eval()
    val_loss, val_correct = 0, 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            preds = outputs.argmax(1)
            val_loss += loss.item() * images.size(0)
            val_correct += (preds == labels).sum().item()
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = val_correct / len(val_loader.dataset)
    val_f1 = f1_score(val_labels, val_preds, average='weighted')

    print(f"Epoch {epoch} | Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    scheduler.step()

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved at epoch {epoch} with Val Acc: {val_acc:.4f}")
    else:
        early_stop_counter += 1
        if early_stop_counter >= PATIENCE:
            print("⛔ Early stopping triggered.")
            break

print(f"✅ Training complete. Best Val Accuracy: {best_val_acc:.4f}")


/tmp/ipykernel_35/2698562640.py:36: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1 [Train]:   0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 [Val]:   0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 [Val]: 100%|██████████| 8/8 [00:01<00:00,  5.90it/s]


Epoch 1 | Train Acc: 0.3760 | Train F1: 0.3628 | Val Acc: 0.8122 | Val F1: 0.8152
✅ Best model saved at epoch 1 with Val Acc: 0.8122


Epoch 2 [Train]:   0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 2 [Val]:   0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 2 [Val]: 100%|██████████| 8/8 [00:01<00:00,  5.60it/s]


Epoch 2 | Train Acc: 0.5624 | Train F1: 0.5599 | Val Acc: 0.8571 | Val F1: 0.8595
✅ Best model saved at epoch 2 with Val Acc: 0.8571


Epoch 3 [Train]:   0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 3 [Val]:   0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 3 [Val]: 100%|██████████| 8/8 [00:01<00:00,  5.75it/s]


...

Epoch 23 | Train Acc: 0.6299 | Train F1: 0.6284 | Val Acc: 0.9755 | Val F1: 0.9756


Epoch 24 [Train]:   0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 24 [Val]:   0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 24 [Val]: 100%|██████████| 8/8 [00:01<00:00,  5.78it/s]


Epoch 24 | Train Acc: 0.6545 | Train F1: 0.6532 | Val Acc: 0.9347 | Val F1: 0.9343


Epoch 25 [Train]:   0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 25 [Val]:   0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_35/2698562640.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 25 [Val]: 100%|██████████| 8/8 [00:01<00:00,  5.90it/s]


Epoch 25 | Train Acc: 0.6508 | Train F1: 0.6505 | Val Acc: 0.9796 | Val F1: 0.9797
✅ Best model saved at epoch 25 with Val Acc: 0.9796
✅ Training complete. Best Val Accuracy: 0.9796
